# Clothing Reviews Neural Network Model with TensorFlow

### Business Understanding

In the fashion industry, product reviews are very important as they can influence consumer purchasing decisions. Therefore, companies often collect and analyze product reviews to understand customer preferences and needs, as well as to improve product quality and services. The goal of this project is to build a neural network model using TensorFlow to perform sentiment analysis on clothing reviews, so that the company can gain deeper insights into customer preferences and improve customer satisfaction.

### Problem Statement
The company wants to understand customer preferences and needs regarding their clothing products through the analysis of existing product reviews. However, the large and heterogeneous amount of reviews poses a challenge in conducting manual analysis. Therefore, a model that can automatically perform sentiment analysis on product reviews is needed to help the company gain deeper insights into customer preferences and needs.

### Solution
The proposed solution is to build a neural network model using TensorFlow, which can automatically perform sentiment analysis on product reviews. The model will be trained using a dataset of clothing product reviews that have been collected by the company, with the goal of classifying reviews into positive, negative, or neutral. By utilizing this model, the company can save time and resources required to manually analyze reviews, as well as gain deeper insights into customer preferences and needs.

### Import Library

In [ ]:
import numpy as np 
import pandas as pd 
from plotly import graph_objs as go

In [ ]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

df = pd.read_csv('Womens Clothing E-Commerce Reviews.csv')
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [ ]:
del df['Unnamed: 0']
print('There are', df.shape[0], 'reviews in this dataset')
print('Number of Duplicates:', len(df[df.duplicated()]))
print('Number of Missing Values:', df.isnull().sum().sum())

There are 23486 reviews in this dataset
Number of Duplicates: 21
Number of Missing Values: 4697


### Cleaning Data

In [ ]:
print('Number of Missing Values per column:')
df.isnull().sum().sort_values(ascending=False)

Number of Missing Values per column:


Title                      3810
Review Text                 845
Division Name                14
Department Name              14
Class Name                   14
Clothing ID                   0
Age                           0
Rating                        0
Recommended IND               0
Positive Feedback Count       0
dtype: int64

Remove missing value in data

In [ ]:
df = df.dropna(subset = ['Review Text', 'Division Name', 'Department Name', 'Class Name'])

In [ ]:
rating = (
    df['Rating']
    .value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={'index':'Rating', 'Rating':'Count'})
    .sort_values(by=['Rating'], ascending=True)   
          ) 

rating['percent'] = ((rating['Count'] / rating['Count'].sum())*100).round(2).astype(str) + '%'

colors = ['#0E2F44',] * 5
colors[4] = '#f6b220'

fig = go.Figure(go.Bar(
            y=rating['Count'],
            x=rating['Rating'],
            marker_color=colors,
            text=rating['percent']
                        ))

fig.update_traces(texttemplate='%{text}', 
                  textposition='outside',
                  cliponaxis = False,
                  hovertemplate='<b>Rating</b>: %{x}<br><extra></extra>'+  
                                '<b>Count</b>: %{y}',
                  textfont_size=18)
                  
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
 
fig.update_layout(coloraxis=dict(colorscale='Teal'),
                  showlegend=False, 
                  plot_bgcolor='#F7F7F7', 
                  margin=dict(pad=20),
                  paper_bgcolor='#F7F7F7',
                  height=500,
                  yaxis={'showticklabels': False},
                  yaxis_title=None,
                  xaxis_title=None,
                  title_text="<b>Rating</b> Distribution",
                  title_x=0.5,
                  font=dict(family="Rockwell, sans-serif", size=22, color='#000000'),
                  title_font_size=35)


fig.show()

## NLP

Import Libraray that will be used in modeling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

import re
import string

from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import SimpleRNN, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
data = df[['Review Text', 'Recommended IND']]

clean the text from some unwanted elements.

In [ ]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, 
    remove links, remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

* text = str(text).lower(): Convert all the letters in text to lowercase.
* text = re.sub('\[.*?\]', '', text): Remove all characters inside square brackets, such as [abc].
* text = re.sub('https?://\S+|www\.\S+', '', text): Remove all links starting with http://, https://, or www., followed by other characters.
* text = re.sub('<.*?>+', '', text): Remove all HTML tags, such as <html>, <head>, and so on.
* text = re.sub('[%s]' % re.escape(string.punctuation), '', text): Remove all punctuation marks from the text, such as periods, commas, question marks, and so on.
* text = re.sub('\n', '', text): Remove all newline characters (\n) from the text.
* text = re.sub('\w*\d\w*', '', text): Remove all words that contain numbers.

In [ ]:
data['Review Text'] = data['Review Text'].apply(lambda x:clean_text(x))

In [ ]:
def roc_auc(predictions,target):
    
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

### Divide the data into train data and test data

In [ ]:
X = data.drop('Recommended IND', axis=1)
y = data['Recommended IND']

# Spliting train & test
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    stratify=y,
                                                    test_size=0.2, 
                                                    random_state=42,
                                                    shuffle=True)

TOKENISASI

In [ ]:
num_words = None   # the most X frequent words is returned
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X_train['Review Text'].tolist() + X_test['Review Text'].tolist())   # introduce text in list
word_index = tokenizer.word_index

X_train_seq = tokenizer.texts_to_sequences(X_train['Review Text'].tolist())
X_test_seq = tokenizer.texts_to_sequences(X_test['Review Text'].tolist())

max_len = max([len(x) for x in X_train_seq])

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

print("\nPadded training shape:", X_train_pad.shape)
print("\nPadded test shape:", X_test_pad.shape)


Padded training shape: (18102, 112)

Padded test shape: (4526, 112)


SEQUENTIAL, EMBEDDING, LSM

In [ ]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                    50,     # embeds it in a 50-dimensional vector
                    input_length=max_len))

model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 112, 50)           1022500   
                                                                 
 lstm_1 (LSTM)               (None, 100)               60400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 1,083,001
Trainable params: 1,083,001
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train_pad, y_train, epochs= 6, batch_size=512,
                    validation_data=(X_test_pad, y_test), 
                    verbose=2)

Epoch 1/6
36/36 - 49s - loss: 0.1281 - accuracy: 0.9515 - val_loss: 0.2927 - val_accuracy: 0.8829 - 49s/epoch - 1s/step
Epoch 2/6
36/36 - 46s - loss: 0.1171 - accuracy: 0.9581 - val_loss: 0.3098 - val_accuracy: 0.8791 - 46s/epoch - 1s/step
Epoch 3/6
36/36 - 47s - loss: 0.1039 - accuracy: 0.9633 - val_loss: 0.3965 - val_accuracy: 0.8847 - 47s/epoch - 1s/step
Epoch 4/6
36/36 - 46s - loss: 0.0960 - accuracy: 0.9664 - val_loss: 0.3420 - val_accuracy: 0.8864 - 46s/epoch - 1s/step
Epoch 5/6
36/36 - 46s - loss: 0.0853 - accuracy: 0.9698 - val_loss: 0.3684 - val_accuracy: 0.8840 - 46s/epoch - 1s/step
Epoch 6/6
36/36 - 46s - loss: 0.0796 - accuracy: 0.9729 - val_loss: 0.3684 - val_accuracy: 0.8776 - 46s/epoch - 1s/step


### Model Evaluation

In [ ]:
model.evaluate(X_test_pad, y_test)

142/142 [==============================] - 4s 30ms/step - loss: 0.3684 - accuracy: 0.8776


[0.3683913052082062, 0.8775961399078369]

The output indicates the result of a machine learning model training process. The training process produced a model with a loss of 0.3684 and an accuracy of 0.8776. These values correspond to the final loss and accuracy of the model after training. The numbers in the first line, 4s 30ms/step, indicate the time required to train one batch of data, where 4s represents the time required for one epoch or one iteration through the entire training data, and 30ms/step represents the time required to train one batch of data in each epoch.

In this case, with an accuracy value of 0.8776, the model can be considered good enough. However, further evaluation needs to be done by considering other factors such as data diversity, dataset size, and others.